In [1]:
import pandas as pd
import numpy as np
import os
import glob
import random

import warnings
warnings.filterwarnings("ignore")

In [2]:
def preprocess_data(data, is_train=True):
    
    temp = data.copy()
    temp = temp[['Hour', 'TARGET', 'DHI', 'DNI', 'WS', 'RH', 'T']]

    if is_train==True:          
    
        temp['Target1'] = temp['TARGET'].shift(-48).fillna(method='ffill')
        temp['Target2'] = temp['TARGET'].shift(-48*2).fillna(method='ffill')
        temp = temp.dropna()
        
        return temp.iloc[:-96]

    elif is_train==False:
        
        temp = temp[['Hour', 'TARGET', 'DHI', 'DNI', 'WS', 'RH', 'T']]
                              
        return temp.iloc[-48:, :]

In [3]:
train = pd.read_csv('./data/train/train.csv')

test = []

for i in range(81):
    file_path = './data/test/' + str(i) + '.csv'
    temp = pd.read_csv(file_path)
    temp = preprocess_data(temp, is_train=False)
    test.append(temp)

df_test = pd.concat(test)

submission = pd.read_csv('./data/sample_submission.csv')

In [4]:
df_train = preprocess_data(train)

df_train.shape, df_test.shape

((52464, 9), (3888, 7))

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

df_train[['Hour','DHI','DNI','WS','RH','T']] = scaler.fit_transform(df_train[['Hour','DHI','DNI','WS','RH','T']])
df_test[['Hour','DHI','DNI','WS','RH','T']] = scaler.fit_transform(df_test[['Hour','DHI','DNI','WS','RH','T']])

In [6]:
Day  = df_train.iloc[:, :-2]
Day7 = df_train.iloc[:, -2]
Day8 = df_train.iloc[:, -1]

In [7]:
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras import layers
from tensorflow.keras.backend import mean, maximum

In [8]:
def quantile_loss(q, y, pred):
    err = (y-pred)
    return mean(maximum(q*err, (q-1)*err), axis=-1)

In [9]:
q_lst = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [10]:
model7 = tf.keras.Sequential([
    layers.Dense(52, activation='relu', input_shape=(Day.shape)),
    layers.Dense(39, activation='relu'),
    layers.Dense(26, activation='relu'),
    layers.Dense(1)
])

model8 = tf.keras.Sequential([
    layers.Dense(52, activation='relu', input_shape=(Day.shape)),
    layers.Dense(39, activation='relu'),
    layers.Dense(26, activation='relu'),
    layers.Dense(1)
])

In [ ]:
from sklearn.model_selection import train_test_split

X_train_1, X_valid_1, Y_train_1, Y_valid_1 = train_test_split(Day, Day7, test_size=0.25, random_state=0)
X_train_2, X_valid_2, Y_train_2, Y_valid_2 = train_test_split(Day, Day8, test_size=0.25, random_state=0)

X_train_1.shape, X_valid_1.shape, Y_train_1.shape, Y_valid_1.shape

In [ ]:
model7.compile(loss='mse', optimizer='adam')
model7.fit(X_train_1, Y_train_1, epochs=25, batch_size=48, validation_split=0.25)
model7.evaluate(X_valid_1, Y_valid_1)

In [ ]:
model8.compile(loss='mse', optimizer='adam')
model8.fit(X_train_2, Y_train_2, epochs=25, batch_size=48, validation_split=0.25)
model8.evaluate(X_valid_2, Y_valid_2)

In [11]:
results7 = pd.DataFrame()

for q in q_lst:
    model7.compile(loss=lambda y,pred: quantile_loss(q,y,pred), optimizer='adam')
    model7.fit(Day, Day7, epochs=25, batch_size=48, validation_split=0.25)
    pred = pd.DataFrame(model7.predict(df_test))
    results7 = pd.concat([results7, pred], axis=1)

results7.shape

Epoch 1/25
820/820 [==============================] - 2s 2ms/step - loss: 1.4416 - val_loss: 1.5999
Epoch 2/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4112 - val_loss: 1.5963
Epoch 3/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4125 - val_loss: 1.5877
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4135 - val_loss: 1.5745
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4084 - val_loss: 1.5686
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3702 - val_loss: 1.5518
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3580 - val_loss: 1.5565
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3667 - val_loss: 1.5548
Epoch 9/25
820/820 [==============================] - 1s 2ms/step - loss: 1.3489 - val_loss: 1.5640
Epoch 10/25
820/820 [==============================] - 2s 2ms/step - loss: 1.3513 - val_loss: 1.5524

820/820 [==============================] - 1s 1ms/step - loss: 2.5251 - val_loss: 2.9783
Epoch 2/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5236 - val_loss: 2.9690
Epoch 3/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4901 - val_loss: 2.9690
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4803 - val_loss: 2.9985
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4867 - val_loss: 2.9486
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4529 - val_loss: 3.0459
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4621 - val_loss: 3.0028
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4697 - val_loss: 3.0105
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4913 - val_loss: 2.9596
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4667 - val_loss: 3.0439
Epoch 11/2

820/820 [==============================] - 1s 1ms/step - loss: 2.4050 - val_loss: 2.9744
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4441 - val_loss: 3.0187
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 2.3863 - val_loss: 2.9698
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4129 - val_loss: 2.9697
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4046 - val_loss: 2.9840
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4168 - val_loss: 2.9730
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 2.3691 - val_loss: 2.9273
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4315 - val_loss: 3.0029
Epoch 11/25
820/820 [==============================] - 1s 1ms/step - loss: 2.3979 - val_loss: 3.0258
Epoch 12/25
820/820 [==============================] - 1s 1ms/step - loss: 2.4322 - val_loss: 2.9472
Epoch 13

Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7726 - val_loss: 2.2498
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7388 - val_loss: 2.2084
Epoch 11/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7725 - val_loss: 2.2634
Epoch 12/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7567 - val_loss: 2.3246
Epoch 13/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7532 - val_loss: 2.2675
Epoch 14/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7421 - val_loss: 2.2627
Epoch 15/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7815 - val_loss: 2.2193
Epoch 16/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7539 - val_loss: 2.2413
Epoch 17/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7199 - val_loss: 2.2521
Epoch 18/25
820/820 [==============================] - 1s 1ms/step - loss: 1.7528 - val_loss

820/820 [==============================] - 1s 1ms/step - loss: 0.6943 - val_loss: 0.8902
Epoch 15/25
820/820 [==============================] - 1s 1ms/step - loss: 0.6950 - val_loss: 0.9194
Epoch 16/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7006 - val_loss: 0.9443
Epoch 17/25
820/820 [==============================] - 1s 1ms/step - loss: 0.6910 - val_loss: 0.9267
Epoch 18/25
820/820 [==============================] - 1s 1ms/step - loss: 0.6850 - val_loss: 0.9140
Epoch 19/25
820/820 [==============================] - 1s 1ms/step - loss: 0.6960 - val_loss: 0.8985
Epoch 20/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7030 - val_loss: 0.9250
Epoch 21/25
820/820 [==============================] - 1s 1ms/step - loss: 0.6932 - val_loss: 0.9498
Epoch 22/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7020 - val_loss: 0.9121
Epoch 23/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7076 - val_loss: 0.9057
Ep

(3888, 9)

In [12]:
results8 = pd.DataFrame()

for q in q_lst:
    model8.compile(loss=lambda y,pred: quantile_loss(q,y,pred), optimizer='adam')
    model8.fit(Day, Day8, epochs=25, batch_size=48, validation_split=0.25)
    pred = pd.DataFrame(model8.predict(df_test))
    results8 = pd.concat([results8, pred], axis=1)

results8.shape

Epoch 1/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4566 - val_loss: 1.6398
Epoch 2/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4526 - val_loss: 1.6274
Epoch 3/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4205 - val_loss: 1.6580
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4123 - val_loss: 1.5880
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4086 - val_loss: 1.5799
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3888 - val_loss: 1.5966
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3890 - val_loss: 1.6017
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4015 - val_loss: 1.5686
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 1.4067 - val_loss: 1.5804
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 1.3838 - val_loss: 1.5578

820/820 [==============================] - 1s 1ms/step - loss: 2.6679 - val_loss: 3.0992
Epoch 2/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5764 - val_loss: 3.0783
Epoch 3/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6566 - val_loss: 3.1451
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6064 - val_loss: 3.0906
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5932 - val_loss: 3.0756
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6197 - val_loss: 3.0775
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6075 - val_loss: 3.1267
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5761 - val_loss: 3.1004
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5989 - val_loss: 3.1174
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6016 - val_loss: 3.0498
Epoch 11/2

820/820 [==============================] - 1s 1ms/step - loss: 2.5330 - val_loss: 3.1461
Epoch 3/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5992 - val_loss: 3.1483
Epoch 4/25
820/820 [==============================] - 1s 1ms/step - loss: 2.6009 - val_loss: 3.1405
Epoch 5/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5456 - val_loss: 3.1605
Epoch 6/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5642 - val_loss: 3.1490
Epoch 7/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5450 - val_loss: 3.1326
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5550 - val_loss: 3.1373
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5934 - val_loss: 3.1832
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5457 - val_loss: 3.1139
Epoch 11/25
820/820 [==============================] - 1s 1ms/step - loss: 2.5408 - val_loss: 3.1609
Epoch 12/

820/820 [==============================] - 1s 1ms/step - loss: 1.8463 - val_loss: 2.3302
Epoch 8/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8705 - val_loss: 2.3226
Epoch 9/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8074 - val_loss: 2.3100
Epoch 10/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8145 - val_loss: 2.3423
Epoch 11/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8217 - val_loss: 2.3080
Epoch 12/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8386 - val_loss: 2.3143
Epoch 13/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8165 - val_loss: 2.3540
Epoch 14/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8099 - val_loss: 2.3577
Epoch 15/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8539 - val_loss: 2.3179
Epoch 16/25
820/820 [==============================] - 1s 1ms/step - loss: 1.8355 - val_loss: 2.3389
Epoc

820/820 [==============================] - 1s 1ms/step - loss: 0.7307 - val_loss: 1.0344
Epoch 13/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7228 - val_loss: 0.9567
Epoch 14/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7200 - val_loss: 0.9443
Epoch 15/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7173 - val_loss: 0.9446
Epoch 16/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7211 - val_loss: 0.9597
Epoch 17/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7374 - val_loss: 0.9823
Epoch 18/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7197 - val_loss: 0.9672
Epoch 19/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7254 - val_loss: 1.0000
Epoch 20/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7254 - val_loss: 0.9355
Epoch 21/25
820/820 [==============================] - 1s 1ms/step - loss: 0.7049 - val_loss: 0.9557
Ep

(3888, 9)

In [13]:
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = results7.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = results8.sort_index().values
submission

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.csv_Day7_0h00m,-0.011723,0.001854,0.006749,0.001154,-0.004049,0.002504,0.002971,0.000585,0.010779
1,0.csv_Day7_0h30m,-0.011741,0.001993,0.007170,0.001356,-0.003754,0.002384,0.002812,0.000675,0.011019
2,0.csv_Day7_1h00m,-0.011242,0.001906,0.007920,0.001324,-0.003835,0.002207,0.002537,0.001064,0.009492
3,0.csv_Day7_1h30m,-0.011212,0.001905,0.007619,0.001364,-0.004121,0.002274,0.002599,0.001041,0.009259
4,0.csv_Day7_2h00m,-0.010999,0.001645,0.007671,0.000544,-0.004563,0.002308,0.002403,0.001141,0.007459
...,...,...,...,...,...,...,...,...,...,...
7771,80.csv_Day8_21h30m,-0.000980,-0.001061,-0.000185,0.000268,0.000720,-0.000013,0.000657,0.000411,0.000757
7772,80.csv_Day8_22h00m,-0.000980,-0.001061,-0.000185,0.000268,0.000720,-0.000013,0.000657,0.000411,0.000757
7773,80.csv_Day8_22h30m,-0.000980,-0.001061,-0.000185,0.000268,0.000720,-0.000013,0.000657,0.000411,0.000757
7774,80.csv_Day8_23h00m,-0.000980,-0.001061,-0.000185,0.000268,0.000720,-0.000013,0.000657,0.000411,0.000757


In [14]:
submission.to_csv('./submission/submission_20210114-1.csv', index=False)